In [12]:
# stream data

import pandas as pd
import numpy as np

import ssl

ssl._create_default_https_context = ssl._create_unverified_context

arrestData = pd.read_csv("PPAD.csv")

nPopulations = pd.read_csv("totalPopulation.csv", index_col = 'Id')

eduA = pd.read_csv("eduAttainment.csv")


#eduA.head(10)

In [13]:
# create dict of neighborhoods w default vals
nc = {}

for i in range(1, len(nPopulations)):
    nValue = nPopulations.loc[i, "Neighborhood"]
    if(nValue not in nc):
        nc[nValue] = 0
print(nc)  

{'Allegheny Center': 0, 'Allegheny West': 0, 'Allentown': 0, 'Arlington': 0, 'Arlington Heights': 0, 'Banksville': 0, 'Bedford Dwellings': 0, 'Beechview': 0, 'Beltzhoover': 0, 'Bloomfield': 0, 'Bluff': 0, 'Bon Air': 0, 'Brighton Heights': 0, 'Brookline': 0, 'California-Kirkbride': 0, 'Carrick': 0, 'Central Business District': 0, 'Central Lawrenceville': 0, 'Central Northside': 0, 'Central Oakland': 0, 'Chartiers City': 0, 'Chateau': 0, 'Crafton Heights': 0, 'Crawford-Roberts': 0, 'Duquesne Heights': 0, 'East Allegheny': 0, 'East Carnegie': 0, 'East Hills': 0, 'East Liberty': 0, 'Elliott': 0, 'Esplen': 0, 'Fairywood': 0, 'Fineview': 0, 'Friendship': 0, 'Garfield': 0, 'Glen Hazel': 0, 'Greenfield': 0, 'Hays': 0, 'Hazelwood': 0, 'Highland Park': 0, 'Homewood North': 0, 'Homewood South': 0, 'Homewood West': 0, 'Knoxville': 0, 'Larimer': 0, 'Lincoln Place': 0, 'Lincoln-Lemington-Belmar': 0, 'Lower Lawrenceville': 0, 'Manchester': 0, 'Marshall-Shadeland': 0, 'Middle Hill': 0, 'Morningside': 

In [14]:
# create dict of neighborhoods : populations

numOfNeighborhoods = 0

for i in range(len(nPopulations)):
    numOfNeighborhoods += 1

pop = {}

for i in range(1, len(nPopulations)):
    val = nPopulations.loc[i, "Neighborhood"]
    if(val in nPopulations.values):
        pop[val] = nPopulations.get_value(nPopulations.loc[nPopulations['Neighborhood'] == val].index[0], 'Estimate; Total')

pop

/ihome/crc/install/python/miniconda3-3.7/lib/python3.7/site-packages/ipykernel_launcher.py:13: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  del sys.path[0]


{'Allegheny Center': 1411,
 'Allegheny West': 343,
 'Allentown': 2558,
 'Arlington': 1852,
 'Arlington Heights': 272,
 'Banksville': 3858,
 'Bedford Dwellings': 1349,
 'Beechview': 8078,
 'Beltzhoover': 1702,
 'Bloomfield': 8669,
 'Bluff': 6294,
 'Bon Air': 920,
 'Brighton Heights': 7421,
 'Brookline': 13160,
 'California-Kirkbride': 803,
 'Carrick': 10122,
 'Central Business District': 4060,
 'Central Lawrenceville': 4775,
 'Central Northside': 2892,
 'Central Oakland': 5822,
 'Chartiers City': 560,
 'Chateau': 3,
 'Crafton Heights': 3964,
 'Crawford-Roberts': 2225,
 'Duquesne Heights': 2522,
 'East Allegheny': 2095,
 'East Carnegie': 610,
 'East Hills': 2859,
 'East Liberty': 5537,
 'Elliott': 2491,
 'Esplen': 309,
 'Fairywood': 1181,
 'Fineview': 1270,
 'Friendship': 1840,
 'Garfield': 3966,
 'Glen Hazel': 716,
 'Greenfield': 7690,
 'Hays': 405,
 'Hazelwood': 4257,
 'Highland Park': 6810,
 'Homewood North': 3371,
 'Homewood South': 2276,
 'Homewood West': 850,
 'Knoxville': 4255,
 '

In [15]:
# create dict of neighborhoods : crimes

for i in range(len(arrestData)):
    nValue = arrestData.loc[i, "INCIDENTNEIGHBORHOOD"]
    if(nValue in nc):
        nc[nValue] += 1
nc   

{'Allegheny Center': 651,
 'Allegheny West': 75,
 'Allentown': 539,
 'Arlington': 168,
 'Arlington Heights': 100,
 'Banksville': 141,
 'Bedford Dwellings': 336,
 'Beechview': 686,
 'Beltzhoover': 313,
 'Bloomfield': 551,
 'Bluff': 783,
 'Bon Air': 125,
 'Brighton Heights': 612,
 'Brookline': 693,
 'California-Kirkbride': 152,
 'Carrick': 1313,
 'Central Business District': 2131,
 'Central Lawrenceville': 272,
 'Central Northside': 23,
 'Central Oakland': 304,
 'Chartiers City': 23,
 'Chateau': 262,
 'Crafton Heights': 293,
 'Crawford-Roberts': 470,
 'Duquesne Heights': 103,
 'East Allegheny': 1235,
 'East Carnegie': 27,
 'East Hills': 580,
 'East Liberty': 995,
 'Elliott': 349,
 'Esplen': 105,
 'Fairywood': 62,
 'Fineview': 496,
 'Friendship': 107,
 'Garfield': 396,
 'Glen Hazel': 69,
 'Greenfield': 279,
 'Hays': 62,
 'Hazelwood': 706,
 'Highland Park': 286,
 'Homewood North': 1036,
 'Homewood South': 1138,
 'Homewood West': 417,
 'Knoxville': 752,
 'Larimer': 452,
 'Lincoln Place': 10

In [16]:
# create dict of neighborhoods : arrest rates (per 100k)

for key, value in nc.items():
    nc[key] = (value / pop[key]) * 100000
nc

{'Allegheny Center': 46137.49114103473,
 'Allegheny West': 21865.889212827988,
 'Allentown': 21071.149335418297,
 'Arlington': 9071.274298056156,
 'Arlington Heights': 36764.705882352944,
 'Banksville': 3654.743390357698,
 'Bedford Dwellings': 24907.33876945886,
 'Beechview': 8492.201039861353,
 'Beltzhoover': 18390.129259694477,
 'Bloomfield': 6355.98108201638,
 'Bluff': 12440.419447092469,
 'Bon Air': 13586.95652173913,
 'Brighton Heights': 8246.86699905673,
 'Brookline': 5265.95744680851,
 'California-Kirkbride': 18929.016189290163,
 'Carrick': 12971.744714483304,
 'Central Business District': 52487.68472906404,
 'Central Lawrenceville': 5696.335078534032,
 'Central Northside': 795.2973720608575,
 'Central Oakland': 5221.573342493988,
 'Chartiers City': 4107.142857142857,
 'Chateau': 8733333.333333332,
 'Crafton Heights': 7391.523713420787,
 'Crawford-Roberts': 21123.595505617977,
 'Duquesne Heights': 4084.06026962728,
 'East Allegheny': 58949.880668257756,
 'East Carnegie': 4426.22

In [17]:
# eliminate neighborhoods above average num of arrests

ncUpdated = {}

NATL_ARREST_RATE = 3152.6

for key, value in nc.items():
    if(value <= NATL_ARREST_RATE):
        ncUpdated[key] = value
ncUpdated

{'Central Northside': 795.2973720608575,
 'Lincoln Place': 2886.539011146042,
 'Mount Oliver Borough': 353.045013239188,
 'New Homestead': 2261.0483042137716,
 'North Oakland': 2322.4328264944807,
 'Regent Square': 1839.3030009680542,
 'Squirrel Hill North': 1332.0395201129147,
 'Squirrel Hill South': 2736.566512903628,
 'Stanton Heights': 2415.458937198068,
 'Swisshelm Park': 1486.988847583643}

In [18]:
# create dict of all neighborhoods : education levels
# sufficient education defined as completion of HS or greater

allEduLevels = {}

for i in range(len(eduA)):
    totalVal = eduA.loc[i, "Estimate; Total:"]
    hsGradNum = eduA.loc[i, "Estimate; Total: - Regular high school diploma"]
    collegeUnderOne = eduA.loc[i, "Estimate; Total: - Some college, less than 1 year"]
    collegeOnePlus = eduA.loc[i, "Estimate; Total: - Some college, 1 or more years, no degree"]
    associates = eduA.loc[i, "Estimate; Total: - Associate's degree"]
    bachelors = eduA.loc[i, "Estimate; Total: - Bachelor's degree"]
    masters = eduA.loc[i, "Estimate; Total: - Master's degree"]
    professional = eduA.loc[i, "Estimate; Total: - Professional school degree"]
    doctorate = eduA.loc[i, "Estimate; Total: - Doctorate degree"]
    totalEdu = hsGradNum + collegeUnderOne + collegeOnePlus + associates + bachelors + masters + professional + doctorate
    neigh = eduA.loc[i, "Neighborhood"]
    if(neigh not in allEduLevels):
        allEduLevels[neigh] = totalEdu / totalVal

allEduLevels

{'Allegheny Center': 0.9111776447105788,
 'Allegheny West': 0.89419795221843,
 'Allentown': 0.771718452758402,
 'Arlington': 0.7955246913580247,
 'Arlington Heights': 0.4793388429752066,
 'Banksville': 0.9001233045622689,
 'Bedford Dwellings': 0.6851351351351351,
 'Beechview': 0.8426756985605419,
 'Beltzhoover': 0.7536231884057971,
 'Bloomfield': 0.9233212341197822,
 'Bluff': 0.5867158671586716,
 'Bon Air': 0.8640132669983416,
 'Brighton Heights': 0.8768573551263001,
 'Brookline': 0.8552031509121062,
 'California-Kirkbride': 0.7715430861723447,
 'Carrick': 0.7929671527872427,
 'Central Business District': 0.9359861591695502,
 'Central Lawrenceville': 0.8611812216052499,
 'Central Northside': 0.8506111362607515,
 'Central Oakland': 0.832810867293626,
 'Chartiers City': 0.7923250564334086,
 'Chateau': 1.0,
 'Crafton Heights': 0.8654587509637626,
 'Crawford-Roberts': 0.7656957928802589,
 'Duquesne Heights': 0.8908554572271387,
 'East Allegheny': 0.7853577371048253,
 'East Carnegie': 0.843

In [19]:
# create dict of select neighborhoods :  education level

selectEduLevels = {}

for key in ncUpdated.keys():
    if(key not in selectEduLevels):
        selectEduLevels[key] = allEduLevels[key]

selectEduLevels

{'Central Northside': 0.8506111362607515,
 'Lincoln Place': 0.8523464326592903,
 'Mount Oliver Borough': 0.7085137085137085,
 'New Homestead': 0.8847795163584637,
 'North Oakland': 0.9298245614035088,
 'Regent Square': 0.9874686716791979,
 'Squirrel Hill North': 0.9797030633339597,
 'Squirrel Hill South': 0.9524178950090509,
 'Stanton Heights': 0.90607424071991,
 'Swisshelm Park': 0.873405299313052}

In [21]:
# find neighborhood with highest education rate

maxVal = max(selectEduLevels.values())

for key, value in selectEduLevels.items():
    if(value == maxVal):
        print("The best neighborhood in Pittsburgh is: " + key)

The best neighborhood in Pittsburgh is: Regent Square
